## Exercise 1.17

Consider the following code, and assume that nvectors is small compared to the cache size, and length large.

```
for (k=0; k<nvectors; k++)
    for (i=0; i<length; i++)
        a[k,i] = b[i] * c[k]
```

How do the following concepts relate to the performance of this code:
- Reuse
- Cache size
- Associativity

Would the following code where the loops are exchanged perform better or worse, and why?

```
for (i=0; i<length; i++)
    for (k=0; k<nvectors; k++)
        a[k,i] = b[i] * c[k]
```

## 1.17 Answer

This code appears to be computing an `nvectors x length` matrix of results using the `b` vector with length `length` and the `c` vector with length `nvectors`. Since the outer iteration variable `k` goes up to `nvectors` and the inner `i` goes to `length`, the results are produced left to right in the matrix, one full row at a time.

Computing `a[0,0]` will induce compulsory cache misses for both `b[0]` and `c[0]`, but for the remainder of the inner loop `c[0]` is reused and ideally would remain in cache. However, this is unlikely to be true since all elements of `b` will be used and since `length` is large compared to the cache size, it is likely `b` cannot entirely fit in cache. Thus, on subsequent iterations even though `b[0]` is reused, it will likely suffer a capacity cache miss at this point. This logic can be extrapolated to the entire `b` vector and therefore the reuse does not help due to the cache size being insufficient.

The problem is also complicated by associativity. For instance, even though `c[k]` is reused in each inner iteration it may get kicked out of cache if an element of `b` maps to the same place in cache as it. Increasing the associativity increases the number of places a cacheline can be mapped to, which mitigates this issue.

The second code with exchanged loops would perform better than the first one. This code computes the results of the `a` matrix column by column. In this case, after we compute the first column the entire `c` vector should be stored in cache since `nvectors` is small compared to the cache size. Thus, when computing subsequent columns the elements of `c` should remain in cache the entire time as well as `b[i]` for each individual computation of that column. Of course, this still depends on having good associativity as poor mapping could prevent making full use of the cache size.